In [193]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
import numpy as np

In [185]:
train = pd.read_csv('../data/train_final.csv')
test = pd.read_csv('../data/test_final.csv')

target_column = "Survived"
id_column = "PassengerId"

features = ["Pclass", "Fare", "TitleClass", "FamilySize", "AgeClass", "SexClass", "EmbarkedClass", "pctSurvived"]

train_Y = train[target_column]
train_X = train[features]

test_X = test[features]

# treinando

In [187]:
anova_filter = SelectKBest(f_regression, k=10)
rf = RandomForestClassifier()
enc = OneHotEncoder(sparse=False, categorical_features=categorical_columns)

anova_rf = Pipeline([\
                     ('hot', enc),
                     ('anova', anova_filter),\
                     ('rf', rf)])

param_grid = {"rf__n_estimators": [10, 50, 100, 400],\
         "rf__min_samples_split": [3, 5, 10],\
         "rf__min_samples_leaf": [3, 5, 10],\
         "rf__criterion": ["gini", "entropy"],\
         "rf__class_weight": [{0:1, 1:1}, {0:4, 1:1}]}


cv = GridSearchCV(anova_rf, param_grid=param_grid)

cv.fit(train_X, train_Y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('hot', OneHotEncoder(categorical_features=array([ True, False,  True, False,  True,  True,  True, False], dtype=bool),
       dtype=<class 'float'>, handle_unknown='error', n_values='auto',
       sparse=False)), ('anova', SelectKBest(k=10, score_func=<function f_regression at 0x000000171711...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'rf__criterion': ['gini', 'entropy'], 'rf__n_estimators': [10, 50, 100, 400], 'rf__min_samples_leaf': [3, 5, 10], 'rf__class_weight': [{0: 1, 1: 1}, {0: 4, 1: 1}], 'rf__min_samples_split': [3, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

# analisando desempenho no train set

In [196]:
print(classification_report(train["Survived"], cv.predict(train_X)))

             precision    recall  f1-score   support

          0       0.95      0.98      0.96       549
          1       0.97      0.91      0.94       342

avg / total       0.96      0.96      0.95       891



# predizendo

In [197]:
predictions = cv.predict(test_X)

submission = pd.DataFrame({target_column: predictions, id_column: test[id_column]})

submission.to_csv("../submissions/Testando algumas tools do scikit.csv", index=False)